## 3. Plot the results of IrrigatedWIT

Load the shapefile that's output from preprocess_irrigatedWIT.ipynb into a GIS and label the polygons by the 'uniqueID' column, this allows for searching for polygons of interest.
Enter the name of a polygon of interest and a stack plot is produced. Set export_plot=True for the plots to be saved as .png files

## User Inputs

In [1]:
# Enter the Name of the polygon you want to check:
# i.e., the 'UniqueID' column in the shapefile
polyName = ''

In [33]:
# Where are the .csv files stored?
filepath = "results/GWYDIR/2018/csvs/"

# do you want to export the plot?
export_plot = False

# If so, where?
output_dir = "results/GWYDIR/2018/plots/"

### Run the cells below to produce a stack plot of the polygon

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

In [ ]:
#import the csv and prepare dataframe
df = pd.read_csv(filepath + polyName + ".csv")
df['Datetime'] = pd.to_datetime(df.Datetime)
df['Datetime'] = df.Datetime.dt.normalize()
df = df.set_index('Datetime')

In [ ]:
# generate plot
warnings.filterwarnings("ignore")
#set up color palette
pal = [sns.xkcd_rgb["cobalt blue"],
       sns.xkcd_rgb["neon blue"],
       sns.xkcd_rgb["grass"],
       sns.xkcd_rgb["beige"],
       sns.xkcd_rgb["brown"]]       

#make a stacked area plot
plt.clf()
fig= plt.figure(figsize = (22,6))
plt.stackplot(df.index, 
              df.wofs_area_percent, 
              df.wet_percent, 
              df.green_veg_percent, 
              df.dry_veg_percent,
              df.bare_soil_percent,
              labels=['open water',
                      'wet',
                      'green veg',
                      'dry veg',
                      'bare soil',
                     ], colors=pal, alpha = 0.6)

#set axis limits to the min and max
plt.axis(xmin = df.index[0], xmax = df.index[-1], ymin = 0, ymax = 100)

#add a legend and a tight plot box
plt.legend(loc='lower left', framealpha=0.6)
plt.title('Fractional Cover, Wetness, and Water for '+polyName)
plt.tight_layout()
if export_plot:
    #save the figure
    plt.savefig(f'{output_dir}{polyName}.png')#, transparent=True)


### Multiproccesing to generate all plots at once (terria-cube workflow)

In [2]:
import os
from multiprocessing import Pool

In [34]:
csvs = os.listdir(filepath)
csvs.sort()
csvs[1]

'GWYDIR_2018_1.csv'

In [35]:
def generate_plots(file):
    #import the csv and prepare dataframe
    df = pd.read_csv(filepath + file)
    polyName = file[:-4]
    df['Datetime'] = pd.to_datetime(df.Datetime)
    df['Datetime'] = df.Datetime.dt.normalize()
    df = df.set_index('Datetime')

    # generate plot
    warnings.filterwarnings("ignore")
    #set up color palette
    pal = [sns.xkcd_rgb["cobalt blue"],
           sns.xkcd_rgb["neon blue"],
           sns.xkcd_rgb["grass"],
           sns.xkcd_rgb["beige"],
           sns.xkcd_rgb["brown"]]       

    #make a stacked area plot
    plt.clf()
    fig= plt.figure(figsize = (22,6))
    plt.stackplot(df.index, 
                  df.wofs_area_percent, 
                  df.wet_percent, 
                  df.green_veg_percent, 
                  df.dry_veg_percent,
                  df.bare_soil_percent,
                  labels=['open water',
                          'wet',
                          'green veg',
                          'dry veg',
                          'bare soil',
                         ], colors=pal, alpha = 0.6)

    #set axis limits to the min and max
    plt.axis(xmin = df.index[0], xmax = df.index[-1], ymin = 0, ymax = 100)

    #add a legend and a tight plot box
    plt.legend(loc='lower left', framealpha=0.6)
    plt.title('Fractional Cover, Wetness, and Water for '+polyName)
    plt.tight_layout()

    #save the figure
    plt.savefig(output_dir + file[:-4] + '.png')#, transparent=True)

In [37]:
pool = Pool(6)    
pool.map(generate_plots, csvs)